In [14]:
import cv2
import numpy as np
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
from tensorflow.keras.models import Sequential, model_from_json

In [15]:
 
# load json and create model
json_file = open('model_bw.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_bw.h5")
print("Loaded model from disk")
 

Loaded model from disk


In [16]:
def detect_face_mask(img):

  y_pred = loaded_model.predict(img.reshape(1,224,224,3),verbose=0)

  return 0 if y_pred[0][0] < 0.6 else 1

In [17]:
sample1 = cv2.resize(cv2.imread('13-with-mask_jpg.rf.3f70707aaf596885be28d4a087396a2f.jpg'),(224,224))

sample1.shape

(224, 224, 3)

In [18]:
print(detect_face_mask(sample1))

0


In [50]:
def draw_label(img,text,pos,bg_color,border_thickness=2, text_thickness=2, padding=10):
    text_size = cv2.getTextSize(text , cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)

    end_x = pos[0] + text_size[0][0] + 2 * padding
    end_y = pos[1] + text_size[0][1] + 2 * padding

    cv2.rectangle(img,pos,(end_x,end_y), bg_color , cv2.FILLED)

    # Adjust text position to be inside the rectangle
    text_x = pos[0] + + padding
    text_y = pos[1] + text_size[0][1] + padding - 2

    
    cv2.putText(img,text,(text_x,text_y),cv2.FONT_HERSHEY_SIMPLEX , 1 , (0,0,0),2,cv2.LINE_AA)

In [51]:
face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def detect_face(img):
    coord = face.detectMultiScale(img)

    return coord

In [53]:
cap = cv2.VideoCapture(0)

while True :

  ret , frame = cap.read()

  face_loc = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

  img = cv2.resize(frame , (224,224))

  y_pred = detect_face_mask(img)

  for x,y,w,h in face_loc :

    cv2.rectangle(frame,(x,y),(x+w, y + h) , (255,0,0),2)
 

  if y_pred == 1:
    draw_label(frame,'MASK',(30,30),(60, 179, 113))
  else :
    draw_label(frame,'NO MASK',(30,30),(0,0,255))


  cv2.imshow('App', frame)


  if cv2.waitKey(1) & 0xFF == ord('x'):
    break

cap.release()
cv2.destroyAllWindows()